# Groq + Browser Use MCP: Intelligent Product Comparison
**The Problem:** Comparing products online means opening multiple tabs, checking various retailers, and manually tracking prices and features across different sources.

**The Solution:** This tutorial shows how to combine Groq's fast inference with Browser Use's Model Context Protocol (MCP) server to automatically research and compare products across multiple websites in real-time.

## What is Browser Use MCP?
Browser Use MCP enables AI models to autonomously browse websites, extract information, and interact with web pages. Think of it as giving your model the ability to:
- Navigate websites and click links
- Extract data from multiple sources
- Compare information across retailers

You can learn more about Browser Use at [browser-use.com](https://browser-use.com).

## Prerequisites

In [17]:
# install dependencies
!pip install openai python-dotenv

## Setup and Configuration

In [18]:
#!/usr/bin/env python3
"""
Groq + Browser Use MCP Demo
"""

import json
import os
import time
from datetime import datetime

from openai import OpenAI

# Set API Keys

Here we will set the API keys for the Groq and Browser Use APIs via either environment variables or a userdata object in Google Colab.

- You can get your Groq API key from [here](https://console.groq.com/keys)
- You can get your Browser Use API key from [here](https://browser-use.com)

In [19]:
try:
    from google.colab import userdata

    GROQ_API_KEY = userdata.get("GROQ_API_KEY")
    BROWSER_USE_API_KEY = userdata.get("BROWSER_USE_API_KEY")
except ImportError:
    # Try loading from .env file first, then fall back to environment
    try:
        from dotenv import load_dotenv
        load_dotenv()
    except ImportError:
        pass  # dotenv not available, will use os.getenv directly
    
    # Get from environment (either from .env or shell environment)
    GROQ_API_KEY = os.getenv("GROQ_API_KEY")
    BROWSER_USE_API_KEY = os.getenv("BROWSER_USE_API_KEY")

# Check if API keys are set
if not GROQ_API_KEY:
    print("Please set your Groq API key")
elif not BROWSER_USE_API_KEY:
    print("Please set your Browser Use API key")
else:
    print("Groq API key and Browser Use API key configured successfully!")

# Model configuration
MODEL = "openai/gpt-oss-120b"

Groq API key and Browser Use API key configured successfully!


In [20]:
def browser_research(query):
    """
    Research products using Groq + Browser Use MCP

    This function demonstrates the speed and accuracy of combining:
    - Groq's fast LLM inference (500+ tokens/second)
    - Browser Use MCP server for real-time web browsing
    """

    if not GROQ_API_KEY:
        print("Please set your Groq API key first!")
        return

    client = OpenAI(base_url="https://api.groq.com/openai/v1", api_key=GROQ_API_KEY, timeout=300)

    print(f"{query}")

    start_time = time.time()

    # Configure MCP tools using Browser Use server
    tools = [
        {
            "type": "mcp",
            "server_url": "https://api.browser-use.com/mcp/",
            "server_label": "browseruse",
            "require_approval": "never",
            "headers": {
                "X-Browser-Use-API-Key": BROWSER_USE_API_KEY
            }
        }
    ]

    try:
        # Call Groq with Browser Use MCP integration - use streaming to avoid timeouts
        response = client.responses.create(
            model=MODEL,
            input=query,
            instructions="You are a helpful research assistant. Use browseruse tools to find accurate, up-to-date information. Keep tasks focused, simple and fast.",
            tools=tools,
            stream=False,
            temperature=0.3,
            top_p=0.8,
            timeout=300
        )

        total_time = time.time() - start_time

        # Get response content from responses API format
        content = (
            response.output[-1].content[0].text if hasattr(response, "output") else str(response)
        )

        print("=" * 80)
        print("BROWSER RESEARCH RESULTS")
        print("=" * 80)
        print(content)
        print("=" * 80)

        # Show executed tools (MCP tool calls) for transparency
        executed_tools = []

        # Extract MCP calls from response if available
        if hasattr(response, "output") and response.output:
            for output_item in response.output:
                if hasattr(output_item, "type") and output_item.type == "mcp_call":
                    executed_tools.append(
                        {
                            "type": "mcp",
                            "arguments": getattr(output_item, "arguments", "{}"),
                            "output": getattr(output_item, "output", ""),
                            "name": getattr(output_item, "name", ""),
                            "server_label": getattr(output_item, "server_label", ""),
                        }
                    )

        if executed_tools:
            print(f"\nBROWSER USE MCP CALLS: Found {len(executed_tools)} tool calls:")
            print("-" * 50)

            for i, tool in enumerate(executed_tools, 1):
                print(f"\nTool Call #{i}")
                print(f"   Type: {tool['type']}")
                print(f"   Tool Name: {tool['name']}")
                print(f"   Server: {tool['server_label']}")
                try:
                    if tool["arguments"]:
                        args = (
                            json.loads(tool["arguments"])
                            if isinstance(tool["arguments"], str)
                            else tool["arguments"]
                        )
                        print(f"   Arguments: {args}")

                    # Print browser results for transparency
                    if tool["output"]:
                        output_data = (
                            json.loads(tool["output"])
                            if isinstance(tool["output"], str)
                            else tool["output"]
                        )
                        print(f"   Output: {str(output_data)}")
                except Exception as e:
                    print(f"   Could not parse tool data: {e}")

        # Performance summary
        print("\nPERFORMANCE SUMMARY")
        print(f"   Total time: {total_time:.2f} seconds")
        print(f"   Browser Use MCP calls: {len(executed_tools)}")
        print(f"   Response length: {len(content)} characters")
        print(
            "Note: Response time includes real-time web browsing and data extraction across multiple sites - this is normal for autonomous web agent interactions!"
        )
        return {
            "content": content,
            "total_time": total_time,
            "timestamp": datetime.now().isoformat(),
        }

    except Exception as e:
        total_time = time.time() - start_time
        error_msg = f"Browser research failed after {total_time:.2f} seconds: {str(e)}"
        print(f"\\n❌ {error_msg}")
        
        return {
            "content": error_msg,
            "total_time": total_time,
            "timestamp": datetime.now().isoformat(),
            "error": True
        }

## Demo 1: Get latest Google Stock Price

Let's fetch real-time Google stock data via browser:

In [21]:
result_stock_price = browser_research(
    "whats the current price of google stock?"
)

whats the current price of google stock?
BROWSER RESEARCH RESULTS
The current price of Alphabet Inc. (Google) stock (ticker **GOOGL**) is **$252.53**.

BROWSER USE MCP CALLS: Found 2 tool calls:
--------------------------------------------------

Tool Call #1
   Type: mcp
   Tool Name: browser_task
   Server: browseruse
   Arguments: {'max_steps': 5, 'task': 'Navigate to https://finance.yahoo.com/quote/GOOGL and extract the current stock price displayed prominently on the page.'}
   Output: {'task_id': '5e97d719-e496-429c-ba14-decdf7d555a3', 'session_id': '3dfb86f4-9fa0-46d3-b6b9-d2a41804ec7a', 'live_url': '', 'status': 'Task created and started successfully.', 'message': 'TASK CREATED: Use get_browser_task_status with task_id="5e97d719-e496-429c-ba14-decdf7d555a3" to check progress. DO NOT create a new task - always check status first. This task may take time to complete.'}...

Tool Call #2
   Type: mcp
   Tool Name: get_browser_task_status
   Server: browseruse
   Arguments: {'task_i

**What you witnessed:** MCP in action - where models autonomously browse websites and extract information during generation. The model doesn't just generate text; it actively navigates web pages and retrieves real-time data.

###  Why This Matters

Browser Use MCP demonstrates true autonomous web browsing. Combined with any tools (APIs, databases, file systems), you can build AI that researches and acts on the web independently.

Ready to build? Let's go.